In [1]:
import json, torch
import lightning  as L
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from torch.nn import functional as F
import csv
from collections import defaultdict
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import numpy as np
import pickle

/home/davidteng/anaconda3/envs/wsd/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Cwb2LocModel(L.LightningModule):
    def __init__(self, input_dim=7, hidden_dim=128, num_layers=2, output_dim=1, learning_rate=1e-3, delta=1.0):
        super(Cwb2LocModel, self).__init__()
        self.save_hyperparameters()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.delta = delta  # 動態設置 delta

        # 定義 LSTM 層
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.2, bidirectional=True)

        # 定義全連接層
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim // 4)
        self.fc2 = nn.Linear(hidden_dim // 4, input_dim)
        self.fc3 = nn.Linear(input_dim, 1)

        # 定義損失函數（Huber Loss）
        # self.criterion = nn.HuberLoss(delta=self.delta)
        # self.criterion = nn.MSELoss()
        self.criterion = nn.L1Loss()
        # self.criterion = log_cosh_loss

    def log_cosh_loss(y_pred, y_true):
        loss = torch.mean(torch.log(torch.cosh(y_pred - y_true)))
        return loss

    def forward(self, x):
        # 初始化隱藏狀態和細胞狀態
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(self.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(self.device)
        residual = x
        
        # 前向傳播 LSTM
        out, _ = self.lstm(x, (h0, c0))

        # 通過全連接層得到最終輸出
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out += residual
        out = self.fc3(out)
        
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.criterion(y_hat, y)  # 使用 Huber Loss
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.criterion(y_hat, y)  # 使用 Huber Loss
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamW(
            self.parameters(),
            lr=self.hparams.learning_rate,
            weight_decay=1e-2
        )
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.9)
        return [optimizer], [scheduler]


In [3]:
from collections import defaultdict
import numpy as np
import csv

# 定义月份和特征
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']
features = ['rain', 'raintime', 'solarpower', 'suntime', 'temp', 'uv']

# 初始化 cwb_data_dict，四层嵌套：特征 -> 月份 -> 天数 -> 时间索引
# 时间索引初始为小时（0-23），插值后为分钟（0-1439）
cwb_data_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(float))))

# 读取气象数据
for month in months:
    for feature in features:
        csv_path = f'../cwbdata/{month}/{feature}-{month}.csv'
        try:
            with open(csv_path, 'r', encoding='utf-8') as csv_file:
                reader = csv.reader(csv_file)
                for row in reader:
                    if row[0].isdigit():
                        day = int(row[0])
                        for hour_index, value in enumerate(row[1:], start=0):
                            if hour_index < 24:
                                try:
                                    cwb_data_dict[feature][int(month)][day][hour_index] = float(value)
                                except (ValueError, TypeError):
                                    cwb_data_dict[feature][int(month)][day][hour_index] = None  # 标记为无效值
        except FileNotFoundError:
            print(f"文件未找到: {csv_path}")

# 插值函数
def interpolate_feature(feature_data):
    """
    对某一天的一个特征进行插值，从小时级别扩展到分钟级别。
    feature_data: dict, {hour: value} 表示该天的一个特征数据
    返回插值后的 1440 个分钟值
    """
    hours = sorted(feature_data.keys())  # 获取所有小时索引
    values = [feature_data[hour] for hour in hours]  # 获取每小时的值

    # 处理无效值：用前后有效值的平均值填充
    for i in range(len(values)):
        if values[i] is None:
            prev_valid = next((values[j] for j in range(i - 1, -1, -1) if values[j] is not None), None)
            next_valid = next((values[j] for j in range(i + 1, len(values)) if values[j] is not None), None)
            if prev_valid is not None and next_valid is not None:
                values[i] = (prev_valid + next_valid) / 2
            elif prev_valid is not None:
                values[i] = prev_valid
            elif next_valid is not None:
                values[i] = next_valid
            else:
                values[i] = 0.0  # 如果前后都没有有效值，填充为 0.0

    # 线性插值
    x = np.array(hours)  # 小时索引
    y = np.array(values)  # 每小时的值
    x_minute = np.linspace(0, 23, 1440)  # 每分钟的索引
    interpolated_values = np.interp(x_minute, x, y)  # 插值到分钟级别

    return interpolated_values

# 对每个特征的每一天数据进行插值并更新到字典中
for feature in features:
    for month in cwb_data_dict[feature]:
        for day in cwb_data_dict[feature][month]:
            # 插值数据
            hourly_data = cwb_data_dict[feature][month][day]
            interpolated_values = interpolate_feature(hourly_data)
            
            # 将插值结果以每小时为单位存储到字典中
            for hour in range(24):
                start_idx = hour * 60
                end_idx = (hour + 1) * 60
                cwb_data_dict[feature][month][day][hour] = interpolated_values[start_idx:end_idx].tolist()

# 验证插值结果
for feature in features:
    for month in cwb_data_dict[feature]:
        for day in cwb_data_dict[feature][month]:
            for hour in cwb_data_dict[feature][month][day]:
                assert len(cwb_data_dict[feature][month][day][hour]) == 60, \
                    f"插值结果错误：{feature}, 月份 {month}, 日期 {day}, 小时 {hour}"
                assert all(isinstance(value, float) for value in cwb_data_dict[feature][month][day][hour]), \
                    f"存在非 float 值：{feature}, 月份 {month}, 日期 {day}, 小时 {hour}"

print((cwb_data_dict['rain'][2][22][23]))


[0.9715079916608751, 0.9635163307852679, 0.9555246699096589, 0.9475330090340517, 0.9395413481584427, 0.9315496872828355, 0.9235580264072265, 0.9155663655316193, 0.9075747046560103, 0.8995830437804031, 0.8915913829047959, 0.8835997220291869, 0.8756080611535797, 0.8676164002779707, 0.8596247394023635, 0.8516330785267545, 0.8436414176511473, 0.8356497567755383, 0.8276580958999311, 0.8196664350243221, 0.8116747741487149, 0.8036831132731059, 0.7956914523974987, 0.7876997915218897, 0.7797081306462825, 0.7717164697706735, 0.7637248088950663, 0.7557331480194573, 0.7477414871438501, 0.7397498262682412, 0.7317581653926339, 0.723766504517025, 0.7157748436414177, 0.7077831827658088, 0.6997915218902016, 0.6917998610145943, 0.6838082001389854, 0.6758165392633781, 0.6678248783877692, 0.659833217512162, 0.651841556636553, 0.6438498957609458, 0.6358582348853368, 0.6278665740097296, 0.6198749131341206, 0.6118832522585134, 0.6038915913829044, 0.5958999305072972, 0.5879082696316882, 0.579916608756081, 0.5

In [4]:
def collate_cwb_data(month, day, loc, features): #0101
    test_x = []
    for hour in range(7, 17):
        for minute in range(60):
            try:
                # 確保索引不超出範圍
                newx = []
                for f in features:
                    newx.append(cwb_data_dict[f][int(month)][int(day)][int(hour)][minute])
                newx.append(loc)
            except Exception as e:
                print(e)
                # 如果出現錯誤，使用默認值
                newx = [0.0] * (len(features) + 1)
            test_x.append(
                newx
            )
    for minute in range(30):
        try:
            # 確保索引不超出範圍
            newx = []
            for f in features:
                newx.append(cwb_data_dict[f][int(month)][int(day)][int(17)][minute])
            newx.append(loc)
        except Exception as e:
            print(e)
            # 如果出現錯誤，使用默認值
            newx = [0.0] * (len(features) + 1)
        test_x.append(
            newx
        )
    return test_x


In [5]:
def get_prediction_at_time(y_hat_original, hour, minute):
    hour = int(hour)
    minute = int(minute)
    if hour < 7 or hour > 18:
        raise ValueError("小時超出範圍，必須在 [7, 18] 之間")
    if minute < 0 or minute > 59:
        raise ValueError("分鐘超出範圍，必須在 [0, 59] 之間")
    index = (hour - 7) * 60 + minute
    
    if index >= len(y_hat_original):
        raise IndexError(f"索引 {index} 超出 y_hat_original 長度 {len(y_hat_original)}")
    # 返回該時間點的預測值
    return list(y_hat_original[index])


In [6]:
model_path = f'./saved_models_v3/best-checkpoint-6-B-best.ckpt'
model = Cwb2LocModel.load_from_checkpoint(model_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)  # 將模型移動到 GPU（如果可用）
model.eval()  # 切換模型到評估模式

Cwb2LocModel(
  (lstm): LSTM(7, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc1): Linear(in_features=256, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=7, bias=True)
  (fc3): Linear(in_features=7, out_features=1, bias=True)
  (criterion): L1Loss()
)

In [7]:
def get_single_sample_output(id, features, model_cls=Cwb2LocModel):
    # 解析 ID
    month = id[4:6]
    day = id[6:8]
    hour = id[8:10]
    minute = id[10:12]
    loc = int(id[12:])
    
    # 加載標準化器
    with open(f'./scalar_v2/x_scaler.pkl', 'rb') as f:
        x_scaler = pickle.load(f)
    with open(f'./scalar_v2/y_scaler.pkl', 'rb') as f:
        y_scaler = pickle.load(f)
    
    # 構建測試數據
    test_x = collate_cwb_data(month, day, loc, features)  # 獲取測試數據 (shape: [max_len, num_features])
    test_x = np.array(test_x, dtype=np.float32)  # 確保數據是 NumPy 格式
    num_samples, num_features = test_x.shape
    test_x = test_x.reshape(-1, num_features)  # 展平成 2D

    # 正規化數據
    test_x_normalized = x_scaler.transform(test_x)  # 對數據進行標準化
    test_x_normalized = test_x_normalized.reshape(1, -1, num_features)  # 添加 batch 維度
    
    # 將數據轉換為 PyTorch 張量並移動到相應設備
    test_x_tensor = torch.tensor(test_x_normalized, dtype=torch.float32).to(device)
    
    # 使用模型進行預測
    with torch.no_grad():  # 禁用梯度計算以加速推理
        y_hat = model(test_x_tensor)  # 模型預測
        y_hat = y_hat.cpu().numpy()  # 將張量轉換為 NumPy 格式
        y_hat = y_hat.reshape(-1, y_hat.shape[-1])  # 展平成 2D
    
    # 反正規化輸出
    y_hat_original = y_scaler.inverse_transform(y_hat)  # 反正規化
    preds = []
    for i in range(int(minute), int(minute)+10):
        preds.append(get_prediction_at_time(y_hat_original, hour, i))
    # print(len(preds))
    return sum([item[0] for item in preds])/10


In [8]:
from tqdm import tqdm
def fill_csv_with_predictions(csv_path, output_csv_path, model_cls, features):
    import pandas as pd

    # 讀取 CSV 文件
    df = pd.read_csv(csv_path)
    
    # 檢查是否包含必要的列
    if '序號' not in df.columns or '答案' not in df.columns:
        raise ValueError("CSV 文件必須包含 '序號' 和 '答案' 列")
    
    # 遍歷每個序號進行預測
    predictions = []
    for seq_id in tqdm(df['序號'], desc="Processing Predictions", unit="sample"):
        try:
            # 使用模型進行預測
            prediction = get_single_sample_output(str(seq_id), features, model_cls=model_cls)
            
            # 確保 prediction 是 float
            if isinstance(prediction, (float, np.float32, np.float64)):
                if prediction >= 0:
                    predictions.append(prediction)
                else:
                    predictions.append(0.0)
            else:
                raise ValueError(f"無效的輸出類型: {type(prediction)}")
        except Exception as e:
            print(f"序號 {seq_id} 預測失敗，錯誤: {e}")
            predictions.append("ERROR")  # 如果失敗則標記為 ERROR
    
    # 將預測結果填入答案列
    df['答案'] = predictions
    
    # 將結果保存為新的 CSV 文件
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    print(f"預測結果已保存到 {output_csv_path}")

In [9]:
features = ['rain', 'raintime', 'solarpower', 'suntime', 'temp', 'uv']

In [13]:
print(get_single_sample_output('20240101172001', features))

-2.1703126


In [11]:
fill_csv_with_predictions('./up.csv', './upB-3.csv', Cwb2LocModel, features)

Processing Predictions:   0%|          | 0/9600 [00:00<?, ?sample/s]

Processing Predictions: 100%|██████████| 9600/9600 [01:02<00:00, 152.90sample/s]


預測結果已保存到 ./upB-3.csv
